In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gzip
import os
import pickle
import platform

import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.misc import imread

from common import * 
import preprocess
import model_io

Using TensorFlow backend.


# Base Model

The full inception model is too slow for real time use on my laptop, so I am just using the first few layers.

Loading the whole inception model is also very slow, so cut out only the weights we need and save them for loading later.

In [ ]:
import inception
import model_io

if not os.path.exists(BASE_MODEL_JSON_FILE):
    model_io.save_model(
        BASE_MODEL_JSON_FILE,
        BASE_MODEL_WEIGHTS_FILE,
        inception.make_cut_model())

In [ ]:
model_io.load_base_model().summary()

# Preprocessing

- Correct the image file paths, since I have several driving logs in different folders.
- Smooth the control inputs. The input from me is quite spikey, because all I can do is press a button or not. This seems like it would make learning quite difficult, so apply smoothing.
- Bottleneck the features to make training reasonably fast.

In [3]:
log = pd.concat([
    preprocess.run('data/ccw_1'),
    preprocess.run('data/cw_1'),
    preprocess.run('data/ccw_2'),
    preprocess.run('data/cw_2'),
    preprocess.run('data/ccw_recover_from_right_1'),
    preprocess.run('data/ccw_recover_from_left_1'),
    preprocess.run('data/cw_recover_from_right_1'),
    preprocess.run('data/cw_recover_from_left_1')
])
print(log.head())

Bottleneck folder exists in data/ccw_1 ; just binding.
Bottleneck folder exists in data/cw_1 ; just binding.
Bottleneck folder exists in data/ccw_2 ; just binding.
Bottleneck folder exists in data/cw_2 ; just binding.
Bottleneck folder exists in data/ccw_recover_from_right_1 ; just binding.
Bottleneck folder exists in data/ccw_recover_from_left_1 ; just binding.
Bottleneck folder exists in data/cw_recover_from_right_1 ; just binding.
Bottleneck folder exists in data/cw_recover_from_left_1 ; just binding.
                                        center_image  \
0  data/ccw_1/IMG/center_2016_12_08_21_38_10_346.jpg   
1  data/ccw_1/IMG/center_2016_12_08_21_38_10_462.jpg   
2  data/ccw_1/IMG/center_2016_12_08_21_38_10_578.jpg   
3  data/ccw_1/IMG/center_2016_12_08_21_38_10_678.jpg   
4  data/ccw_1/IMG/center_2016_12_08_21_38_10_779.jpg   

                                        left_image  \
0  data/ccw_1/IMG/left_2016_12_08_21_38_10_346.jpg   
1  data/ccw_1/IMG/left_2016_12_08_21_38_10_46

## Dataset Summary


In [4]:
def summarize_dataset():
    groups = log.groupby('dataset')
    totals = groups['center_image'].agg([len])
    mean_speed = groups['speed'].mean()
    mean_steering_angle = groups['steering_angle'].mean()
    return pd.concat([totals, mean_speed, mean_steering_angle], axis=1)
summarize_dataset()

,len,speed,steering_angle
dataset,,,
ccw_1,1284,22.558356,-0.043736
ccw_2,2882,8.833225,-0.041838
ccw_recover_from_left_1,1405,11.775736,0.050748
ccw_recover_from_right_1,1361,10.212416,-0.123761
cw_1,850,28.594668,0.018874
cw_2,2391,9.972545,0.036800
cw_recover_from_left_1,1550,10.484323,0.128920
cw_recover_from_right_1,1721,8.972610,0.017962


## Smoothing

I tried two types of smoothing:

- Exponential smoothing: an exponential weighted moving average
- Gaussian smoothing: a 1D gaussian filter

Both seem to track pretty well, but the gaussian smoothing is better at preserving the mean steering angle.

In [ ]:
def plot_smooth_steering_angle():
    df = log[['time', 'steering_angle', 'smooth_steering_angle_1', 'smooth_steering_angle_gaussian_3']]
    df = df[:600]
    df = df.set_index(['time'])
    df.plot(figsize=(12, 6))
plot_smooth_steering_angle()

## Steering Angle Bias

We don't want a lot of bias in the training data. Or maybe this isn't that important... I don't really know.

In [ ]:
print('raw steering angle bias:', np.mean(log['steering_angle']))
print('exponentially smoothed steering angle bias:', np.mean(log['smooth_steering_angle_1']))
print('gaussian smoothed steering angle bias:', np.mean(log['smooth_steering_angle_gaussian_5']))

## Bottleneck Features

In [ ]:
def plot_bottleneck_output(num_images, num_channels, image_stride=50):
    fig, axes = plt.subplots(num_images, num_channels,
                             figsize=(2*num_channels, 2*num_images),
                             sharex=True, sharey=True)
    for i in range(num_images):
        for j in range(num_channels):
            index = i * image_stride
            image = np.load(log['bottleneck_features'].values[i])['center_image']
            axes[i][j].imshow(image[:,:,j], cmap='gray')
plot_bottleneck_output(3,5)

## Smoothed Steering Video

In [ ]:
def steering_angle_to_radians(steering_angle):
    """
    The recorded steering angle ranges from -1 to 1.
    In the sim, it gives values from -25 degrees to 25 degrees.
    """
    return math.radians(steering_angle * 25.0)

def draw_steering_angle(image, steering_angle, color, thickness=2):
    bottom_x = image.shape[1] / 2.0
    bottom_y = image.shape[0]
    radius = bottom_y / 2.0
    steering_angle_radians = steering_angle_to_radians(steering_angle)
    top_x = bottom_x + radius * math.sin(steering_angle_radians)
    top_y = bottom_y - radius * math.cos(steering_angle_radians)
    bottom_x, bottom_y = int(round(bottom_x)), int(round(bottom_y))
    top_x, top_y = int(round(top_x)), int(round(top_y))
    cv2.line(image, (bottom_x, bottom_y), (top_x, top_y), color, thickness)

def make_steering_movie(
    filestem,
    log,
    smoothed_steering_angle_column,
    predicted_steering_angle_column=None):
    
    if platform.system() == 'Linux':
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        filename = filestem + '.avi'
    else:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')        
        filename = filestemp + '.mp4'
    
    input_color = [0, 0, 255]
    smoothed_color = [0, 255, 0]
    predicted_color = [255, 0, 0]
    video = cv2.VideoWriter(filename, fourcc, 10, IMAGE_SHAPE[0:2][::-1])
    for i in range(len(log)):
        if i % 50 == 0:
            print('frame', i)
        image = imread(log['center_image'].values[i])
        steering_angle = log['steering_angle'].values[i]
        draw_steering_angle(image, steering_angle, input_color)

        smoothed_steering_angle = log[smoothed_steering_angle_column].values[i]
        draw_steering_angle(image, smoothed_steering_angle, smoothed_color)
        
        if predicted_steering_angle_column is not None:
            predicted_steering_angle = log[predicted_steering_angle_column].values[i]
            draw_steering_angle(image, predicted_steering_angle, predicted_color)

        cv2.putText(image, log['center_image'].values[i],
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0)
            
        video.write(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    video.release()
    
for dataset in log['dataset'].unique():
    make_steering_movie(os.path.join('data', dataset, 'check'),
                        log[log['dataset'] == dataset][0:100],
                        'smooth_steering_angle_1')
    break

# Model

- I put a 1x1 convolution up front to reduce the depth, in order to make the model a more manageable size.

## Experiments with Unsmoothed Steering Angles

- Tried just adding a linear model (32--128 units, L2 regularization with weights 0.001--0.01) after the 1x1 convolution, but validation loss generally bottomed out at about MSE 0.04, which is too high. It also tended to diverge, at least with the Adam optimizer.
- Tried adding a hidden layer (32-128 units) with ReLU activation and dropout before the linear layer, but that seemed to just predict a small constant steering angle most of the time. Maybe a "dying relus" problem. It seems you can get an MSE around 0.05 by just predicting a constant near zero all the time.
- So I tried tanh activation instead, but that did not seem to help.
- I switched back to L2 regularization with ReLUs, which did a bit better but still understeered.
- I think the problem is that it's too easy to just ignore my occasional steering inputs (key presses).

## Experiments with Smoothed Steering Angles

- Tried the linear model again. No longer predicting constants, but it starts overfitting (increase in validation loss) after only a couple of epochs. Probably going to need something more expressive.
- Tried adding hidden layer (32 units) with ReLU activation and L2 regularization (weight 0.01). Still mostly predicting a small constant.
- Tried tanh activation on the hidden layer. Now it seems much happier: getting to the bridge and MSE 0.015 with nb_filter=64, nb_hidden=32 and l2_weight=0.01. Still understeering a bit on gentler bends.
- Tried reducing L2 weight to 0.005. That seemed to slow down training quite a lot; got to MSE 0.017 after 16 epochs. Still understeering and weaving a bit.
- Tried mean absolute error instead of mean squared error, since all of the errors are actually pretty small. New record on first run: made it to the turnout but didn't turn left. MAE 0.0611. Some weaving, but much less understeer. Much better fit in the steering prediction plot; good to see that better fit corresponds to better performance in this case. Adam optimizer diverged after 3 epochs, however.
- Tried increasing regularization back to 0.01. That seems to make it get stuck on a constant again.
- Tried putting regularization back to 0.005 but using Adagrad instead of Adam. Still getting stuck. Subsequent runs with Adam also getting stuck. Looks like I just got lucky that first time.

In [5]:
import model

In [ ]:
steering_model = model.build(
    input_shape=np.load(log['bottleneck_features'].values[0])['center_image'].shape,
    nb_filter=64,
    nb_hidden=32,
    l2_weight=0.005,
    optimizer='adam'
#     drop_prob=0.5
)

In [ ]:
history = model.train(
    steering_model, log,
    label_column='smooth_steering_angle_gaussian_3',
    test_size=0.2,
    nb_epoch=20,
    batch_size=128,
    side_camera_bias=None,
    save_stem='model')

In [ ]:
# load model with best validation loss
steering_model = model_io.load_model('model.json', 'model.h5')

In [ ]:
log['predicted_steering_angle'] = steering_model.predict_generator(
    model.generate_data(log),
    val_samples=len(log)
).flatten()
print('raw MSE',
    ((log['steering_angle'] - log['predicted_steering_angle'])**2).mean())
print('smoothed MSE',
    ((log['smooth_steering_angle_gaussian_3'] - log['predicted_steering_angle'])**2).mean())

In [ ]:
def plot_predicted_steering_angle():
    df = log[['time', 'steering_angle', 'smooth_steering_angle_gaussian_3', 'predicted_steering_angle']]
    df = df[:600]
    df = df.set_index(['time'])
    df.plot(figsize=(12, 6))
plot_predicted_steering_angle()

In [ ]:
make_steering_movie('test', log[0:500], 'smooth_steering_angle_1', 'predicted_steering_angle')

In [6]:
import itertools

# From http://stackoverflow.com/a/40623158/2053820
def dict_product(dicts):
    """
    >>> list(dict_product(dict(number=[1,2], character='ab')))
    [{'character': 'a', 'number': 1},
     {'character': 'a', 'number': 2},
     {'character': 'b', 'number': 1},
     {'character': 'b', 'number': 2}]
    """
    return (dict(zip(dicts, x)) for x in itertools.product(*dicts.values()))

def make_model_key_stem(key):
    stem = os.path.join('models', 'grid_model_')
    for param, value in key.items():
        if value is None: value = ''
        stem += param + '-' + str(value)
    return stem

In [7]:
def search():
    results_file = 'grid.pickle'
    if os.path.isfile(results_file):
        with open(results_file, 'rb') as f:
            results = pickle.load(f)
    else:
        results = {}
    
    keys = {
        'version': [2,3,4],
        'nb_epoch': [30],
        'side_camera_bias': [None],
        'label_column': ['smooth_steering_angle_gaussian_3'],
        'batch_size': [128],
        'nb_filter': [64, 128],
        'nb_hidden': [64, 32],
        'l2_weight': [0.001, 0.005, 0.01],
        'optimizer': ['adam', 'adagrad']
    }
    
    input_shape = np.load(log['bottleneck_features'].values[0])['center_image'].shape
        
    for key in dict_product(keys):
        print(key)
        frozen_key = frozenset(key.items())
        if frozen_key in results:
            continue
        print('Running...')
        
        key_stem = make_model_key_stem(key)
        model_json = key_stem + '.json'
        model_weights_h5 = key_stem + '.h5'
        
        key = key.copy()
        del key['version']
        
        steering_model = model.build(
            input_shape,
            nb_filter=key.pop('nb_filter'),
            nb_hidden=key.pop('nb_hidden'),
            l2_weight=key.pop('l2_weight'),
            optimizer=key.pop('optimizer'))
        
        history = model.train(
            steering_model,
            log,
            label_column=key.pop('label_column'),
            test_size=0.2,
            nb_epoch=key.pop('nb_epoch'),
            batch_size=key.pop('batch_size'),
            side_camera_bias=key.pop('side_camera_bias'),
            save_stem=key_stem)
        
        steering_model.load_weights(model_weights_h5) # best weights
        
        model_io.save_model(model_json, model_weights_h5, steering_model)
                    
        results[frozen_key] = {
            'history': history.history,
            'model_json': model_json,
            'model_weights_h5': model_weights_h5
        }
        
        with open(results_file, 'wb') as f:
            pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

    return results  

grid = search()

{'nb_hidden': 64, 'label_column': 'smooth_steering_angle_gaussian_3', 'l2_weight': 0.001, 'nb_epoch': 30, 'batch_size': 128, 'nb_filter': 64, 'side_camera_bias': None, 'optimizer': 'adam', 'version': 2}
Running...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 17, 37, 64)    16448       convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 40256)         0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 64)            2576448     flatten_1[0][0]                  
_______________________________________________________________________________

KeyboardInterrupt: 

TODO

- there is a wiggle in ccw_recover_from_left_1 at about 0:42

In [ ]:
def summarize_grid(grid, val_loss_threshold):
    """
    Print out the more promising hyperparameters from a search, according to
    validation loss.
    """
    best_val_loss = 1e9
    best_params = None
    items = sorted(grid, key = lambda k: min(grid[k]['history']['val_loss']))
    for frozen_key in items:
        value = grid[frozen_key]
        key = dict(frozen_key)
        val_loss = value['history']['val_loss']
        min_val_loss = min(val_loss)
        nb_epochs = len(val_loss)
        if min_val_loss < val_loss_threshold:
            print(key, min_val_loss, nb_epochs)
            print('rm -f model.json model.h5')
            print('cp', value['model_json'], 'model.json')
            print('cp', value['model_h5'], 'model.h5')
            print('python drive.py model.json')
            print()
        if min_val_loss < best_val_loss:
            best_val_loss = min_val_loss
            best_params = key
    print('BEST:', best_params, best_val_loss)
summarize_grid(grid, 0.005)

In [ ]:
results_file = 'grid.pickle'
if os.path.isfile(results_file):
    with open(results_file, 'rb') as f:
        grid = pickle.load(f)
